In [2]:
import pandas as pd
import numpy as np 
from enum import Enum
from util import get_split_cols, get_split_frame

In [3]:
train = pd.read_csv("/data/project2/train.csv")
test = pd.read_csv("/data/project2/test.csv")

In [10]:
num_train_df = get_split_frame(train)[0]
num_test_df = get_split_frame(test)[0]
y = num_train_df["Total Household Income"]
num_train_df = num_train_df.drop(["Total Household Income"], axis=1)
#num_train_df = num_train_df.fillna(0)

In [303]:
class NeuronType(Enum):
    INPUT_TO_HIDDEN = 0
    OUTPUT = 1
    HIDDEN = 2
    
    

class Neuron(object):
    def __init__(self, weights, use_activation = True, neuron_type = NeuronType.HIDDEN):
        self.output = None
        self.deltas = None
        self.sum_w_h = None
        self.weights = np.array(weights)
        self.use_activation = use_activation
        self.neuron_type = neuron_type
class NN(object):
    def __init__(self, num_input_nodes, num_hidden_nodes, 
                 num_output_nodes, num_hidden_layers, eta,
                 use_activation_on_output = False, activation_name = "relu"):
        self.outputs = None
        self.activation_name = activation_name
        self.sum_error = 0
        self.eta = eta
        self.num_hidden_layers = num_hidden_layers
        self.num_hidden_nodes = num_hidden_nodes
        self.num_output_nodes = num_output_nodes
        # hidden layer1 -> input to hidden
        self.network = [self.create_randomized_layer_section(num_input_nodes, 
                                                             num_hidden_nodes, 
                                                             False, 
                                                             NeuronType.INPUT_TO_HIDDEN)]
        # remaining hidden layers -> hidden to hidden
        self.network += [self.create_randomized_layer_section(num_hidden_nodes, 
                                                              num_hidden_nodes) for _ in range(num_hidden_layers - 1)]
        # output -> hidden to output
        self.network += [self.create_randomized_layer_section(num_hidden_nodes, 
                                                              num_output_nodes, 
                                                              use_activation_on_output, 
                                                              NeuronType.OUTPUT)]

    def create_randomized_layer_section(self, num_inputs, num_nodes, use_activation = True, neuron_type = NeuronType.HIDDEN):
        return [Neuron(np.random.rand(num_inputs), use_activation, neuron_type) for idx in range(num_nodes)]
    
    def activation(self, sum_w_h):
        if self.activation_name == "sigmoid":
            return 1/ (1 + np.exp(-np.int64(sum_w_h)))
        if self.activation_name == "relu":
            return sum_w_h if sum_w_h > 0 else 0
        
    def d_activation(self, sum_w_h):
        if self.activation_name == "sigmoid":
            return sum_w_h - sum_w_h**2
        if self.activation_name == "relu":
            return 1 if sum_w_h > 0 else 0
        
    def calc_sum_w_h(self, neuron, prev_outputs):
        return np.dot(neuron.weights,prev_outputs)
    
    def calc_neuron_output(self, neuron, prev_outputs):
        neuron.sum_w_h = self.calc_sum_w_h(neuron, prev_outputs)
        neuron.output = self.activation(neuron.sum_w_h) if neuron.use_activation else neuron.sum_w_h
        return neuron
        
    def calc_new_layer_outputs (self, layer, prev_outputs, expected = None): 
        return [self.calc_neuron_output(neuron, prev_outputs) for neuron in layer]
    
    def forward(self, input_layer, flag=False):
        if flag:
            self.network = [input_layer] + self.network
        for layer_index in range(len(self.network) - 1):
            prev_outputs = np.array(self.network[layer_index]) if type(self.network[layer_index][0]) == int\
            or type(self.network[layer_index][0]) == np.int64\
            or type(self.network[layer_index][0]) == np.float128 else np.array([neuron.output for neuron in self.network[layer_index]]) 
            self.network[layer_index + 1] = self.calc_new_layer_outputs(self.network[layer_index + 1], prev_outputs)
#             for neuron in self.network[layer_index + 1]:
#                 print(neuron.output)
    
    def calc_delta(self, neuron, inputs, expected, prev_layer, neuron_index = None):
        if neuron_index != None:
#             print(neuron_index)
#             print(prev_layer[0].__dict__)
            prev_deltas = np.array([prev_neuron.deltas[neuron_index] for prev_neuron in prev_layer]) 
        if neuron.neuron_type == NeuronType.OUTPUT:
#             print("output set")
#             print(neuron.weights)
#             print(expected)
#             print(neuron.output)
            neuron.deltas = np.array(neuron.weights*(neuron.output - expected), dtype=np.float128)
        elif neuron.neuron_type == NeuronType.HIDDEN:
            neuron.deltas = neuron.weights*self.d_activation(neuron.sum_w_h)
        else:
#             print("should hit")
#             print(inputs)
#             print(self.d_activation(neuron.sum_w_h))
            #act = self.d_activation(neuron.sum_w_h)

            neuron.deltas = np.array(inputs*self.d_activation(neuron.sum_w_h), dtype=np.float128)
#         print(neuron.neuron_type)
#         print(prev_deltas)
            if len(neuron.deltas) == 0:
                neuron.deltas = np.zeros(len(inputs))
        if neuron.neuron_type != NeuronType.OUTPUT:
            neuron.deltas = np.array([sum(delta*prev_deltas) for delta in neuron.deltas], dtype=np.float128)
#         print("start")
#         print(neuron.weights)
        neuron.weights = neuron.weights - self.eta*neuron.deltas
        #print(neuron.weights)
        return neuron

    def backward(self, inputs, expected):
        for layer_index in reversed(range(1,len(self.network))):
            if layer_index == len(self.network) - 1:
                #print(self.network[layer_index][0].output)
                self.network[layer_index] = [self.calc_delta(neuron, inputs, expected, None) for neuron in self.network[layer_index]]
            else:
                prev_layer = self.network[layer_index + 1]
                cur_layer = self.network[layer_index]
                self.network[layer_index] = [self.calc_delta(cur_layer[neuron_index], inputs, 
                                                             expected, prev_layer, 
                                                             neuron_index) for neuron_index in range(len(cur_layer))]
                    
                    
    def train2(self):
        data = [[1, 1],[2, 2],[3, 3],[4, 4],[5, 5],[6, 6],[7, 7],[8, 8],[9, 9],[10, 10],[11, 11],[12, 12],[13, 13],[14, 14],[15, 15],[16, 16],[17, 17],[18, 18],[19, 19],[20, 20],[21, 21],[22, 22],[23, 23],[24, 24],[25, 25],[26, 26],[27, 27],[28, 28],[29, 29],[30, 30],[31, 31],[32, 32],[33, 33],[34, 34],[35, 35],[36, 36],[37, 37],[38, 38],[39, 39],[40, 40],[41, 41],[42, 42],[43, 43],[44, 44],[45, 45],[46, 46],[47, 47],[48, 48],[49, 49],[50, 50],[51, 51],[52, 52],[53, 53],[54, 54],[55, 55],[56, 56],[57, 57],[58, 58],[59, 59],[60, 60],[61, 61],[62, 62],[63, 63],[64, 64],[65, 65],[66, 66],[67, 67],[68, 68],[69, 69],[70, 70],[71, 71],[72, 72],[73, 73],[74, 74],[75, 75],[76, 76],[77, 77],[78, 78],[79, 79],[80, 80],[81, 81],[82, 82],[83, 83],[84, 84],[85, 85],[86, 86],[87, 87],[88, 88],[89, 89],[90, 90],[91, 91],[92, 92],[93, 93],[94, 94],[95, 95],[96, 96],[97, 97],[98, 98],[99, 99],[100, 100]]
        expected = [2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32,34,36,38,40,42,44,46,48,50,52,54,56,58,60,62,64,66,68,70,72,74,76,78,80,82,84,86,88,90,92,94,96,98,100,102,104,106,108,110,112,114,116,118,120,122,124,126,128,130,132,134,136,138,140,142,144,146,148,150,152,154,156,158,160,162,164,166,168,170,172,174,176,178,180,182,184,186,188,190,192,194,196,198,200]
        data = np.array(data, dtype=np.float128)
        
        for epoch in range(100):
            self.sum_error = 0
            for i in range(100):
                #print((i+1) + (epoch*10))
                self.network = [data[i]] + self.network
                self.forward(data[i])
                self.backward(data[i], expected[i])
                self.network.pop(0)

    def train(self, train, y, n_epoch):
        for epoch in range(n_epoch):
            self.sum_error = 0
            for row, expected in zip(train, y):
                #print(row)
                self.network = [row] + self.network
                self.forward(row)
                self.backward(row, expected)
                self.network.pop(0)
                self.sum_error += (self.network[-1][0].output - expected) ** 2
            print(self.network[-1][0].output, self.sum_error)
                
    def predict(self, x):
        self.forward(x)
        print(self.network[-1][0].output)
    
    #def calc_
            
#         self.outputs = outputs
#         if expected is not None:
#             self.sum_error += sum((expected - outputs)**2)




In [306]:
n = NN(num_input_nodes = 2, num_hidden_nodes = 10, 
       num_output_nodes = 1, num_hidden_layers = 10, 
       eta = 0.0000000001, use_activation_on_output = False, 
       activation_name="sigmoid")

n.train2()
##n.forward([2,2], True)
#n.backward([2,2], 4)
n.predict([2,2])

6.87734471088


In [215]:
val = []
for i in range(1,101):
    
print(val.strip(",") + "]")

val = "["
for i in range(1,101):
    val += str(i*2) + ","
print(val.strip(",") + "]")

IndentationError: expected an indented block (<ipython-input-215-040c376adc22>, line 4)

In [ ]:
x = num_train_df.head().as_matrix()
yy = y.head().as_matrix()

n = NN(num_input_nodes = len(num_train_df.columns), num_hidden_nodes = len(num_train_df.columns) + 1, 
       num_output_nodes = 1, num_hidden_layers = 10, 
       eta = 0.000001, use_activation_on_output = False, 
       activation_name="sigmoid")
n.train(x, yy, 20)

In [242]:
np.seterr(all="print")

{'divide': 'print', 'invalid': 'print', 'over': 'raise', 'under': 'print'}